### Exploration

In [1]:
!git clone https://github.com/openai/baselines.git
%cd baselines
!pip install -e .
!pip install stable-baselines
!pip install --upgrade tensorflow==1.14.0
!pip install --upgrade tensorflow-gpu==1.14.0
!pip install gym
!pip install pyarrow
!pip install scipy
!pip uninstall --yes tensorboard-plugin-wit
%cd ..

fatal: destination path 'baselines' already exists and is not an empty directory.
/project/to-run-for-results/unsw-nb15/baselines
Obtaining file:///project/to-run-for-results/unsw-nb15/baselines
     |████████████████████████████████| 1.6 MB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 60.5 MB 58.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 50.3 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 48.8 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.15.7-py3-none-any.whl size=1648839 sha256=440db15e4fd9f50277aa9857b4951260b626447a59314268067fefb728c1f923
  Stored in directory: /home/rppeeter/.cache/pip/wheels/be/72/05/d3dfcfc2a31bbf886112b6373881bdf2e9e00d2c943f3b4f91
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=ef21010a1b24d14245c3d5c4723dc8f5ae218fe5efc6799bd21e30f38010eef4
  Stored in directory: /home/rppeeter/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229b

In [2]:
import os
import gym
import time
import json
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from stable_baselines import A2C
from stable_baselines import bench
from stable_baselines import logger
import tensorflow as tf

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4158825239869866903,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11432388534800849765
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2104839420534647524
 physical_device_desc: "device: XLA_GPU device"]

In [3]:
nslkdd_train = pd.read_feather("/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_training-set.feather")
nslkdd_test = pd.read_feather("/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_testing-set.feather")

nslkdd_train = pd.get_dummies(nslkdd_train,columns=['proto' ,'service', 'state'])
nslkdd_test = pd.get_dummies(nslkdd_test, columns=['proto' ,'service', 'state'])
nslkdd_train.head(5)

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_FIN,state_INT,state_REQ,state_RST
0,0.000011,2,0,496,0,90909.09375,254,0,180363632.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,0.000008,2,0,1762,0,125000.00000,254,0,881000000.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2,0.000005,2,0,1068,0,200000.00000,254,0,854400000.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,0.000006,2,0,900,0,166666.65625,254,0,600000000.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,0.000010,2,0,2126,0,100000.00000,254,0,850400000.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
nslkdd_train.info()
nslkdd_train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53946 entries, 0 to 53945
Columns: 192 entries, dur to state_RST
dtypes: bool(3), category(1), float32(11), int16(11), int32(3), int64(2), int8(10), uint8(151)
memory usage: 13.3 MB


,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_FIN,state_INT,state_REQ,state_RST
count,53946.000000,53946.000000,53946.000000,5.394600e+04,5.394600e+04,53946.000000,53946.000000,53946.000000,5.394600e+04,5.394600e+04,...,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000,53946.000000
mean,1.022188,25.869759,25.967060,1.131112e+04,1.985872e+04,36171.152344,150.903681,136.375728,4.417149e+07,9.582494e+05,...,0.000445,0.003782,0.000556,0.000074,0.000019,0.124143,0.691859,0.161495,0.022393,0.000019
std,3.918624,161.154040,141.707459,2.072230e+05,1.865964e+05,112012.312500,106.750997,117.269586,1.947156e+08,2.900805e+06,...,0.021088,0.061379,0.023576,0.008611,0.004305,0.329747,0.461730,0.367990,0.147959,0.004305
min,0.000000,1.000000,0.000000,2.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.004238,4.000000,2.000000,5.320000e+02,1.780000e+02,24.097402,31.000000,29.000000,8.323655e+03,1.787040e+03,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.388905,10.000000,8.000000,9.000000e+02,3.540000e+02,65.447559,254.000000,252.000000,4.005692e+04,5.700335e+03,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.937146,16.000000,18.000000,2.158000e+03,1.824000e+03,3289.241089,254.000000,252.000000,9.414949e+05,2.485340e+05,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,59.999989,10646.000000,11018.000000,1.435577e+07,1.465753e+07,1000000.000000,255.000000,253.000000,5.268000e+09,2.082111e+07,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [5]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    print(col)
    cols = df.columns[:-1]
    print(cols)
    vc = df[col].value_counts()
    print(vc)
    n = vc.iloc[-1]
    print(n)
    m = vc.iloc[0]
    print(m)
    print(int(m-n))
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    print(initial_cut.index)
    df = df.drop(index=initial_cut.index)
    vc = df[col].value_counts()
    print(vc)
    print(int(n*train_frac))
    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [6]:
def nslkdd_split_df(train_df, test_df):
    train_col = train_df.columns[-1]
    print(train_col)
    train_cols = train_df.columns[:-1]
    print(train_cols)
    test_col = test_df.columns[-1]
    print(test_col)
    test_cols = test_df.columns[:-1]
    print(test_cols)

    return train_df[train_cols], train_df[train_col], test_df[test_cols], test_df[test_col]

In [7]:
print(nslkdd_train['attack_cat'].value_counts())
nslkdd_train.drop('label', inplace=True, axis=1)
nslkdd_train['attack_cat'] = nslkdd_train['attack_cat'].astype('object')
atk_idx = nslkdd_train.loc[nslkdd_train['attack_cat'] != "normal"].index
nslkdd_train.loc[atk_idx, 'attack_cat'] = 1.0
nslkdd_train.loc[nslkdd_train.index.difference(atk_idx), 'attack_cat'] = 0.0
nslkdd_train['attack_cat'] = nslkdd_train['attack_cat'].astype(dtype=np.float32)

normal            34200
exploits           7126
fuzzers            4394
generic            3640
reconnaissance     2469
dos                1315
shellcode           377
analysis            312
backdoor             70
worms                43
Name: attack_cat, dtype: int64


In [8]:
print(nslkdd_test['attack_cat'].value_counts())
nslkdd_test.drop('label', inplace=True, axis=1)
nslkdd_test['attack_cat'] = nslkdd_test['attack_cat'].astype('object')
atk_idx = nslkdd_test.loc[nslkdd_test['attack_cat'] != "normal"].index
nslkdd_test.loc[atk_idx, 'attack_cat'] = 1.0
nslkdd_test.loc[nslkdd_test.index.difference(atk_idx), 'attack_cat'] = 0.0
nslkdd_test['attack_cat'] = nslkdd_test['attack_cat'].astype(dtype=np.float32)

normal            51815
exploits          18721
fuzzers           14889
reconnaissance     6422
generic            3924
dos                3004
shellcode          1089
analysis            542
backdoor            513
worms               121
Name: attack_cat, dtype: int64


In [9]:
extra_removables = nslkdd_test.columns ^ nslkdd_train.columns
for to_remove in extra_removables:
            try:
                nslkdd_test.drop(to_remove, inplace=True, axis=1)
            except:
                print(f"{to_remove} already not part of test set, skipping")
            try:
                nslkdd_train.drop(to_remove, inplace=True, axis=1)
            except:
                print(f"{to_remove} already not part of train set, skipping")
nslkdd_train.info()
nslkdd_test.info()

proto_icmp already not part of train set, skipping
proto_rtp already not part of train set, skipping
state_ACC already not part of test set, skipping
state_CLO already not part of test set, skipping
state_ECO already not part of train set, skipping
state_PAR already not part of train set, skipping
state_URN already not part of train set, skipping
state_no already not part of train set, skipping
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53946 entries, 0 to 53945
Columns: 189 entries, dur to state_RST
dtypes: bool(2), float32(12), int16(11), int32(3), int64(2), int8(10), uint8(149)
memory usage: 13.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101040 entries, 0 to 101039
Columns: 189 entries, dur to state_RST
dtypes: bool(2), float32(12), int16(11), int32(3), int64(2), int8(10), uint8(149)
memory usage: 25.0 MB


In [10]:
x_train, y_train, x_test, y_test  = nslkdd_split_df(nslkdd_train, nslkdd_test)

state_RST
Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'sload', 'dload',
       ...
       'service_pop3', 'service_radius', 'service_smtp', 'service_snmp',
       'service_ssh', 'service_ssl', 'state_CON', 'state_FIN', 'state_INT',
       'state_REQ'],
      dtype='object', length=188)
state_RST
Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'sload', 'dload',
       ...
       'service_pop3', 'service_radius', 'service_smtp', 'service_snmp',
       'service_ssh', 'service_ssl', 'state_CON', 'state_FIN', 'state_INT',
       'state_REQ'],
      dtype='object', length=188)


In [11]:
y_train = y_train.to_frame()
y_test = y_test.to_frame()

# custom keys -> replace by index

x_train = x_train.set_index([pd.Index(range (0, len(x_train)))])
y_train = y_train.set_index([pd.Index(range (0, len(y_train)))])
x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])


### Custom environment

In [12]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        self.action_space = gym.spaces.Discrete(2)
         # All the features we have, len(important_features) - 1 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(len(nslkdd_train.columns) - 1,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)
        current_label = self.expected_action
        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {'label': current_label}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()
        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

Train classifier using dqn

In [13]:
def ids_ddqn():
    env = IdsEnv(images_per_episode=1)
    env = bench.Monitor(env, logger.get_dir())

    model = A2C(
        MlpPolicy, 
        env, 
        verbose=1,
    )
    model.learn(
        total_timesteps=int(1.0e5),
        log_interval=10000,
        
    )

    env.close()
    
    return model

start_time = time.time()
ddqn_model_1 = ids_ddqn()
print()
print("A2C 1 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_2 = ids_ddqn()
print()
print("A2C 2 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_3 = ids_ddqn()
print()
print("A2C 3 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_4 = ids_ddqn()
print()
print("A2C 4 Training Time:", time.time() - start_time)


Wrapping the env in a DummyVecEnv.




Instructions for updating:
Use keras.layers.flatten instead.



/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.2      |
| explained_variance | -0.583   |
| fps                | 13       |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| total_timesteps    | 5        |
| value_loss         | 0.493    |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 1        |
| explained_variance | nan      |
| fps                | 286      |
| nupdates           | 10000    |
| policy_entropy     | 0.00135  |
| total_timesteps    | 50000    |
| value_loss         | 3.84e-05 |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 1        |
| e

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.4      |
| explained_variance | -0.035   |
| fps                | 17       |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| total_timesteps    | 5        |
| value_loss         | 0.704    |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 1        |
| explained_variance | nan      |
| fps                | 299      |
| nupdates           | 10000    |
| policy_entropy     | 0.00435  |
| total_timesteps    | 50000    |
| value_loss         | 0.000896 |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 1        |
| explained_variance | nan      |
| fps                | 300      |
| nupdates           | 20000    |
| policy_entropy     | 0.000612 |
| total_timesteps    | 100000   |
| value_loss         | 3.64e-05 |
--------------

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.6      |
| explained_variance | -2.59    |
| fps                | 17       |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| total_timesteps    | 5        |
| value_loss         | 2.75     |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 1        |
| explained_variance | nan      |
| fps                | 297      |
| nupdates           | 10000    |
| policy_entropy     | 0.00242  |
| total_timesteps    | 50000    |
| value_loss         | 6.52e-05 |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 1        |
| explained_variance | nan      |
| fps                | 297      |
| nupdates           | 20000    |
| policy_entropy     | 0.000303 |
| total_timesteps    | 100000   |
| value_loss         | 2.38e-06 |
--------------

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.4      |
| explained_variance | -1.77    |
| fps                | 16       |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| total_timesteps    | 5        |
| value_loss         | 1.35     |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 1        |
| explained_variance | nan      |
| fps                | 297      |
| nupdates           | 10000    |
| policy_entropy     | 0.000932 |
| total_timesteps    | 50000    |
| value_loss         | 5.67e-06 |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 1        |
| explained_variance | nan      |
| fps                | 297      |
| nupdates           | 20000    |
| policy_entropy     | 0.00128  |
| total_timesteps    | 100000   |
| value_loss         | 1.27e-05 |
--------------

In [14]:
# 0 is benign (positive), 1 is malicious (negative) 
def ids_eval(model):
    TP, FP, TN, FN = 0,0,0,0
    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    obs, done = env.reset(), False
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, info = env.step(model.predict(obs)[0])
                label = info['label']
                if label == 0 and rew > 0:
                    TP += 1
                if label == 0 and rew == 0:
                    FP += 1
                if label == 1 and rew > 0:
                    TN += 1
                if label == 1 and rew == 0:
                    FN += 1

    except StopIteration:
        accuracy = (float(TP + TN) / (TP + FP + FN + TN)) 
        precision = (float(TP) / (TP + FP))
        recall = (float(TP) / (TP + FN)) # = TPR = Sensitivity
        FPR = (float(FP) / (TN + FP)) # 1 - specificity
        f1_score = 2 * (precision * recall) / (precision + recall)
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format(accuracy * 100))
        print('Precision: {0}%'.format(precision * 100))
        print('Recall/TPR/Sensitivity: {0}%'.format(recall * 100))
        print('FPR: {0}%'.format(FPR * 100))
        print('F1 score: {0}'.format(f1_score))
    return [accuracy, precision, recall, FPR, f1_score]

models = [ddqn_model_1, ddqn_model_2, ddqn_model_3, ddqn_model_4]

results_1 = ids_eval(ddqn_model_1)
results_2 = ids_eval(ddqn_model_2)
results_3 = ids_eval(ddqn_model_3)
results_4 = ids_eval(ddqn_model_4)
total_results = [-1,-1,-1,-1,-1]
accuracies = [results_1[0], results_2[0], results_3[0], results_4[0]]

for i in range(len(results_1)):
    total_results[i] = (results_1[i] + results_2[i] + results_3[i] + results_4[i] )/ 4

print()    
print('Total validation done...')
print('Accuracy: {0}%'.format(total_results[0] * 100))
print('Precision: {0}%'.format(total_results[1] * 100))
print('Recall/TPR/Sensitivity: {0}%'.format(total_results[2] * 100))
print('FPR: {0}%'.format(total_results[3] * 100))
print('F1 score: {0}'.format(total_results[4]))

highest = 0
for i in range(4):
    if accuracies[i] > accuracies[highest]:
        highest = i

print()
print("Saving model {0}".format(highest + 1))
models[highest].save('a2c_cicddos2019_allfeat.pkl')



/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 99.91290405589976%
Precision: 99.9920759127558%
Recall/TPR/Sensitivity: 99.92081559932693%
FPR: 100.0%
F1 score: 0.9995643305542903

validation done...
Accuracy: 99.90498624279974%
Precision: 99.9841518255116%
Recall/TPR/Sensitivity: 99.9208093286611%
FPR: 100.0%
F1 score: 0.999524705416378

validation done...
Accuracy: 99.91290405589976%
Precision: 99.9920759127558%
Recall/TPR/Sensitivity: 99.92081559932693%
FPR: 100.0%
F1 score: 0.9995643305542903

validation done...
Accuracy: 99.89706842969971%
Precision: 99.9762277382674%
Recall/TPR/Sensitivity: 99.92080305700199%
FPR: 100.0%
F1 score: 0.9994850771394054

Total validation done...
Accuracy: 99.90696569607475%
Precision: 99.98613284732265%
Recall/TPR/Sensitivity: 99.92081089607923%
FPR: 100.0%
F1 score: 0.9995346109160911

Saving model 1
